In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("dataset/student_scores.csv")

df = df.fillna(value=np.nan) # 缺失值補nan

In [3]:
# 轉換Label
label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])
df['part_time_job'] = label_encoder.fit_transform(df['part_time_job'])
df['extracurricular_activities'] = label_encoder.fit_transform(df['extracurricular_activities'])

df

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,1,0,3,0,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,0,0,2,0,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,0,0,9,1,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,0,0,5,0,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,1,0,5,0,10,Unknown,84,77,65,65,80,74,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,1,0,2,0,30,Construction Engineer,83,77,84,73,75,84,82
1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,1,0,2,0,20,Software Engineer,89,65,73,80,87,67,73
1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,0,0,5,0,14,Software Engineer,97,85,63,93,68,94,78
1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,0,1,10,1,5,Business Owner,51,96,72,89,95,88,75


In [4]:
df = pd.get_dummies(df, columns=['career_aspiration'], prefix='career')
career_cols = df.select_dtypes(include=['bool']).columns
df[career_cols] = df[career_cols].astype(int)

df


,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,math_score,...,career_Game Developer,career_Government Officer,career_Lawyer,career_Real Estate Developer,career_Scientist,career_Software Engineer,career_Stock Investor,career_Teacher,career_Unknown,career_Writer
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,1,0,3,0,27,73,...,0,0,1,0,0,0,0,0,0,0
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,0,0,2,0,47,90,...,0,0,0,0,0,0,0,0,0,0
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,0,0,9,1,13,81,...,0,1,0,0,0,0,0,0,0,0
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,0,0,5,0,3,71,...,0,0,0,0,0,0,0,0,0,0
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,1,0,5,0,10,84,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,1,0,2,0,30,83,...,0,0,0,0,0,0,0,0,0,0
1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,1,0,2,0,20,89,...,0,0,0,0,0,1,0,0,0,0
1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,0,0,5,0,14,97,...,0,0,0,0,0,1,0,0,0,0
1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,0,1,10,1,5,51,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.drop("first_name", axis=1, inplace=True)
df.drop("last_name", axis=1, inplace=True)
df.drop("email", axis=1, inplace=True)

df

,id,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,math_score,history_score,physics_score,chemistry_score,...,career_Game Developer,career_Government Officer,career_Lawyer,career_Real Estate Developer,career_Scientist,career_Software Engineer,career_Stock Investor,career_Teacher,career_Unknown,career_Writer
0,1,1,0,3,0,27,73,81,93,97,...,0,0,1,0,0,0,0,0,0,0
1,2,0,0,2,0,47,90,86,96,100,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,9,1,13,81,97,95,96,...,0,1,0,0,0,0,0,0,0,0
3,4,0,0,5,0,3,71,74,88,80,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,5,0,10,84,77,65,65,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,1,0,2,0,30,83,77,84,73,...,0,0,0,0,0,0,0,0,0,0
1996,1997,1,0,2,0,20,89,65,73,80,...,0,0,0,0,0,1,0,0,0,0
1997,1998,0,0,5,0,14,97,85,63,93,...,0,0,0,0,0,1,0,0,0,0
1998,1999,0,1,10,1,5,51,96,72,89,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from sklearn.preprocessing import MinMaxScaler

score_cols = ['math_score', 'history_score', 'physics_score', 'chemistry_score', 'biology_score', 'english_score', 'geography_score']

scaler = MinMaxScaler()

df[score_cols] = scaler.fit_transform(df[score_cols])

df

,id,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,math_score,history_score,physics_score,chemistry_score,...,career_Game Developer,career_Government Officer,career_Lawyer,career_Real Estate Developer,career_Scientist,career_Software Engineer,career_Stock Investor,career_Teacher,career_Unknown,career_Writer
0,1,1,0,3,0,27,0.550000,0.62,0.86,0.94,...,0,0,1,0,0,0,0,0,0,0
1,2,0,0,2,0,47,0.833333,0.72,0.92,1.00,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,9,1,13,0.683333,0.94,0.90,0.92,...,0,1,0,0,0,0,0,0,0,0
3,4,0,0,5,0,3,0.516667,0.48,0.76,0.60,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,5,0,10,0.733333,0.54,0.30,0.30,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,1,0,2,0,30,0.716667,0.54,0.68,0.46,...,0,0,0,0,0,0,0,0,0,0
1996,1997,1,0,2,0,20,0.816667,0.30,0.46,0.60,...,0,0,0,0,0,1,0,0,0,0
1997,1998,0,0,5,0,14,0.950000,0.70,0.26,0.86,...,0,0,0,0,0,1,0,0,0,0
1998,1999,0,1,10,1,5,0.183333,0.92,0.44,0.78,...,0,0,0,0,0,0,0,0,0,0


In [7]:
behavior_cols = ['absence_days', 'weekly_self_study_hours']

df[behavior_cols] = scaler.fit_transform(df[behavior_cols])

df

,id,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,math_score,history_score,physics_score,chemistry_score,...,career_Game Developer,career_Government Officer,career_Lawyer,career_Real Estate Developer,career_Scientist,career_Software Engineer,career_Stock Investor,career_Teacher,career_Unknown,career_Writer
0,1,1,0,0.3,0,0.54,0.550000,0.62,0.86,0.94,...,0,0,1,0,0,0,0,0,0,0
1,2,0,0,0.2,0,0.94,0.833333,0.72,0.92,1.00,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0.9,1,0.26,0.683333,0.94,0.90,0.92,...,0,1,0,0,0,0,0,0,0,0
3,4,0,0,0.5,0,0.06,0.516667,0.48,0.76,0.60,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0.5,0,0.20,0.733333,0.54,0.30,0.30,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,1,0,0.2,0,0.60,0.716667,0.54,0.68,0.46,...,0,0,0,0,0,0,0,0,0,0
1996,1997,1,0,0.2,0,0.40,0.816667,0.30,0.46,0.60,...,0,0,0,0,0,1,0,0,0,0
1997,1998,0,0,0.5,0,0.28,0.950000,0.70,0.26,0.86,...,0,0,0,0,0,1,0,0,0,0
1998,1999,0,1,1.0,1,0.10,0.183333,0.92,0.44,0.78,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv(f"./dataset/student_scores_clean.csv", index=False)